### vgg_model

In [1]:
""" Load VGGNet weights needed for the implementation of the paper 
"A Neural Algorithm of Artistic Style" by Gatys et al. in TensorFlow.

Author: Chip Huyen (huyenn@stanford.edu)
Prepared for the class CS 20SI: "TensorFlow for Deep Learning Research"
For more details, please read the assignment handout:
http://web.stanford.edu/class/cs20si/assignments/a2.pdf
"""

import numpy as np
import tensorflow as tf
import scipy.io

def _weights(vgg_layers, layer, expected_layer_name):
    """ Return the weights and biases already trained by VGG
    """
    W = vgg_layers[0][layer][0][0][2][0][0]
    b = vgg_layers[0][layer][0][0][2][0][1]
    layer_name = vgg_layers[0][layer][0][0][0][0]
    assert layer_name == expected_layer_name
    return W, b.reshape(b.size)

def _conv2d_relu(vgg_layers, prev_layer, layer, layer_name):
    """ Return the Conv2D layer with RELU using the weights, biases from the VGG
    model at 'layer'.
    Inputs:
        vgg_layers: holding all the layers of VGGNet
        prev_layer: the output tensor from the previous layer
        layer: the index to current layer in vgg_layers
        layer_name: the string that is the name of the current layer.
                    It's used to specify variable_scope.

    Output:
        relu applied on the convolution.

    Note that you first need to obtain W and b from vgg-layers using the function
    _weights() defined above.
    W and b returned from _weights() are numpy arrays, so you have
    to convert them to TF tensors using tf.constant.
    Note that you'll have to do apply relu on the convolution.
    Hint for choosing strides size: 
        for small images, you probably don't want to skip any pixel
    """
    with tf.variable_scope(layer_name) as scope:
        W, b = _weights(vgg_layers, layer, layer_name)
        W = tf.constant(W, name='weights')
        b = tf.constant(b, name='bias')
        conv2d = tf.nn.conv2d(prev_layer, filter=W, strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv2d + b)

def _avgpool(prev_layer):
    """ Return the average pooling layer. The paper suggests that average pooling
    actually works better than max pooling.
    Input:
        prev_layer: the output tensor from the previous layer

    Output:
        the output of the tf.nn.avg_pool() function.
    Hint for choosing strides and kszie: choose what you feel appropriate
    """
    return tf.nn.avg_pool(prev_layer, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], 
                          padding='SAME', name='avg_pool_')

def load_vgg(path, input_image):
    """ Load VGG into a TensorFlow model.
    Use a dictionary to hold the model instead of using a Python class
    """
    vgg = scipy.io.loadmat(path)
    vgg_layers = vgg['layers']

    graph = {} 
    graph['conv1_1']  = _conv2d_relu(vgg_layers, input_image, 0, 'conv1_1')
    graph['conv1_2']  = _conv2d_relu(vgg_layers, graph['conv1_1'], 2, 'conv1_2')
    graph['avgpool1'] = _avgpool(graph['conv1_2'])
    graph['conv2_1']  = _conv2d_relu(vgg_layers, graph['avgpool1'], 5, 'conv2_1')
    graph['conv2_2']  = _conv2d_relu(vgg_layers, graph['conv2_1'], 7, 'conv2_2')
    graph['avgpool2'] = _avgpool(graph['conv2_2'])
    graph['conv3_1']  = _conv2d_relu(vgg_layers, graph['avgpool2'], 10, 'conv3_1')
    graph['conv3_2']  = _conv2d_relu(vgg_layers, graph['conv3_1'], 12, 'conv3_2')
    graph['conv3_3']  = _conv2d_relu(vgg_layers, graph['conv3_2'], 14, 'conv3_3')
    graph['conv3_4']  = _conv2d_relu(vgg_layers, graph['conv3_3'], 16, 'conv3_4')
    graph['avgpool3'] = _avgpool(graph['conv3_4'])
    graph['conv4_1']  = _conv2d_relu(vgg_layers, graph['avgpool3'], 19, 'conv4_1')
    graph['conv4_2']  = _conv2d_relu(vgg_layers, graph['conv4_1'], 21, 'conv4_2')
    graph['conv4_3']  = _conv2d_relu(vgg_layers, graph['conv4_2'], 23, 'conv4_3')
    graph['conv4_4']  = _conv2d_relu(vgg_layers, graph['conv4_3'], 25, 'conv4_4')
    graph['avgpool4'] = _avgpool(graph['conv4_4'])
    graph['conv5_1']  = _conv2d_relu(vgg_layers, graph['avgpool4'], 28, 'conv5_1')
    graph['conv5_2']  = _conv2d_relu(vgg_layers, graph['conv5_1'], 30, 'conv5_2')
    graph['conv5_3']  = _conv2d_relu(vgg_layers, graph['conv5_2'], 32, 'conv5_3')
    graph['conv5_4']  = _conv2d_relu(vgg_layers, graph['conv5_3'], 34, 'conv5_4')
    graph['avgpool5'] = _avgpool(graph['conv5_4'])
    
    return graph

### googlenet_model

In [2]:
from tensorflow.contrib.slim.nets import inception
import tensorflow.contrib.slim as slim

def load_googlenet(path, input_image):
    # Load GoogLeNet into a TensorFlow model.
    with slim.arg_scope(inception.inception_v3_arg_scope()):
        logits, end_points = inception.inception_v3(input_image, num_classes=1001, is_training=False)
    return end_points

### utils

In [3]:
""" Utils needed for the implementation of the paper "A Neural Algorithm of Artistic Style"
by Gatys et al. in TensorFlow.

Author: Chip Huyen (huyenn@stanford.edu)
Prepared for the class CS 20SI: "TensorFlow for Deep Learning Research"
For more details, please read the assignment handout:
http://web.stanford.edu/class/cs20si/assignments/a2.pdf
"""
from __future__ import print_function

import os

from PIL import Image, ImageOps
import numpy as np
import scipy.misc
import tarfile
from six.moves import urllib

# VGG-19 parameters file
VGG_DOWNLOAD_LINK = 'http://www.vlfeat.org/matconvnet/models/imagenet-vgg-verydeep-19.mat'
VGG_MODEL = 'imagenet-vgg-verydeep-19.mat'
EXPECTED_BYTES = 534904783

# GoogLeNet parameters file
GOOGLENET_DOWNLOAD_LINK = 'http://download.tensorflow.org/models/inception_v3_2016_08_28.tar.gz'
GOOGLENET_MODEL = 'inception_v3.ckpt'
GOOGLENET_EXPECTED_BYTES = 100885009

def download(download_link, file_name, expected_bytes, model_type):
    """ Download the pretrained model if it's not already downloaded """
    if os.path.exists(file_name):
        print(model_type + " pre-trained model ready")
        return
    print("Downloading the " + model_type + " pre-trained model. This might take a while ...")
    suffix = download_link.rsplit('/', 1)[1]
    urllib.request.urlretrieve(download_link, suffix)
    file_stat = os.stat(suffix)
    if file_stat.st_size == expected_bytes:
        print('Successfully downloaded ' + model_type + ' pre-trained model', suffix)
    else:
        raise Exception('File ' + suffix +
                        ' might be corrupted. You should try downloading it with a browser.')
    if model_type == 'googlenet':
        tar = tarfile.open(suffix)
        tar.extractall()
        tar.close()
        os.remove(suffix)

def get_resized_image(img_path, height, width, save=True):
    image = Image.open(img_path)
    # it's because PIL is column major so you have to change place of width and height
    # this is stupid, i know
    image = ImageOps.fit(image, (width, height), Image.ANTIALIAS)
    if save:
        image_dirs = img_path.split('/')
        image_dirs[-1] = 'resized_' + image_dirs[-1]
        out_path = '/'.join(image_dirs)
        if not os.path.exists(out_path):
            image.save(out_path)
    image = np.asarray(image, np.float32)
    return np.expand_dims(image, 0)

def generate_noise_image(content_image, height, width, noise_ratio=0.6):
    noise_image = np.random.uniform(-20, 20, 
                                    (1, height, width, 3)).astype(np.float32)
    return noise_image * noise_ratio + content_image * (1 - noise_ratio)

def save_image(path, image):
    # Output should add back the mean pixels we subtracted at the beginning
    image = image[0] # the image
    image = np.clip(image, 0, 255).astype('uint8')
    scipy.misc.imsave(path, image)
    
def make_dir(path):
    """ Create a directory if there isn't one already. """
    try:
        os.mkdir(path)
    except OSError:
        pass

def load_model(model_type, input_image):
    global STYLE_LAYERS, CONTENT_LAYER
    if model_type == 'vgg':
        print('Loading VGG-19')
        STYLE_LAYERS = ['conv1_1', 'conv2_1', 'conv3_1', 'conv4_1', 'conv5_1']
        CONTENT_LAYER = 'conv4_2'
        download(VGG_DOWNLOAD_LINK, VGG_MODEL, EXPECTED_BYTES, 'vgg')
        return load_vgg(VGG_MODEL, input_image)
    if model_type == 'googlenet':
        print('Loading GoogLeNet')
        STYLE_LAYERS = ['Conv2d_1a_3x3', 'Conv2d_2a_3x3', 'Conv2d_3b_1x1', 'Conv2d_4a_3x3']
        CONTENT_LAYER = 'Conv2d_2b_3x3'
        download(GOOGLENET_DOWNLOAD_LINK, GOOGLENET_MODEL, GOOGLENET_EXPECTED_BYTES, 'googlenet')
        return load_googlenet(GOOGLENET_MODEL, input_image)
    raise ValueError('No such model type')

### style_transfer

In [4]:
""" An implementation of the paper "A Neural Algorithm of Artistic Style"
by Gatys et al. in TensorFlow.

Author: Chip Huyen (huyenn@stanford.edu)
Prepared for the class CS 20SI: "TensorFlow for Deep Learning Research"
For more details, please read the assignment handout:
http://web.stanford.edu/class/cs20si/assignments/a2.pdf
"""
from __future__ import print_function

import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import time

import numpy as np
import tensorflow as tf

import shutil

# parameters to manage experiments
STYLE = 'dali'
CONTENT = 'max'
STYLE_IMAGE = 'styles/' + STYLE + '.jpg'
CONTENT_IMAGE = 'content/' + CONTENT + '.jpg'
IMAGE_HEIGHT = 299
IMAGE_WIDTH = 299
NOISE_RATIO = 0.6 # percentage of weight of the noise for intermixing with the content image

CONTENT_WEIGHT = 0.1
STYLE_WEIGHT = 0.5

W = [0.5, 1.0, 1.5, 3.0, 4.0] # give more weights to deeper layers.

ITERS = 1000

MEAN_PIXELS = np.array([123.68, 116.779, 103.939]).reshape((1,1,1,3))
""" MEAN_PIXELS is defined according to description on their github:
https://gist.github.com/ksimonyan/211839e770f7b538e2d8
'In the paper, the model is denoted as the configuration D trained with scale jittering. 
The input images should be zero-centered by mean pixel (rather than mean image) subtraction. 
Namely, the following BGR values should be subtracted: [103.939, 116.779, 123.68].'
"""

GOOGLENET = 'googlenet'
VGG = 'vgg'

# model to run (GOOGLENET | VGG)
MODEL_TYPE = GOOGLENET

LR = 0.01 if MODEL_TYPE == GOOGLENET else 2.0

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

def _create_content_loss(p, f):
    """ Calculate the loss between the feature representation of the
    content image and the generated image.
    
    Inputs: 
        p, f are just P, F in the paper 
        (read the assignment handout if you're confused)
        Note: we won't use the coefficient 0.5 as defined in the paper
        but the coefficient as defined in the assignment handout.
    Output:
        the content loss

    """
    return tf.reduce_sum((f - p) ** 2) / (4.0 * p.size)

def _gram_matrix(F, N, M):
    """ Create and return the gram matrix for tensor F
        Hint: you'll first have to reshape F
    """
    F = tf.reshape(F, (M, N))
    return tf.matmul(tf.transpose(F), F)

def _single_style_loss(a, g):
    """ Calculate the style loss at a certain layer
    Inputs:
        a is the feature representation of the real image
        g is the feature representation of the generated image
    Output:
        the style loss at a certain layer (which is E_l in the paper)

    Hint: 1. you'll have to use the function _gram_matrix()
        2. we'll use the same coefficient for style loss as in the paper
        3. a and g are feature representation, not gram matrices
    """
    N = a.shape[3] # number of filters
    M = a.shape[1] * a.shape[2] # height times width of the feature map
    A = _gram_matrix(a, N, M)
    G = _gram_matrix(g, N, M)
    return tf.reduce_sum((G - A) ** 2 / ((2 * N * M) ** 2))

def _create_style_loss(A, model):
    """ Return the total style loss
    """
    n_layers = len(STYLE_LAYERS)
    E = [_single_style_loss(A[i], model[STYLE_LAYERS[i]]) for i in range(n_layers)]
    
    return sum([W[i] * E[i] for i in range(n_layers)])

def _create_losses(model, input_image, content_image, style_image):
    with tf.variable_scope('loss') as scope:
        with tf.Session() as sess:
            if (MODEL_TYPE == GOOGLENET):
                saver = tf.train.Saver(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='InceptionV3'))
                saver.restore(sess, GOOGLENET_MODEL)
            sess.run(input_image.assign(content_image)) # assign content image to the input variable
            p = sess.run(model[CONTENT_LAYER])
        content_loss = _create_content_loss(p, model[CONTENT_LAYER])

        with tf.Session() as sess:
            if (MODEL_TYPE == GOOGLENET):
                saver = tf.train.Saver(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='InceptionV3'))
                saver.restore(sess, GOOGLENET_MODEL)
            sess.run(input_image.assign(style_image))
            A = sess.run([model[layer_name] for layer_name in STYLE_LAYERS])                              
        style_loss = _create_style_loss(A, model)
        total_loss = CONTENT_WEIGHT * content_loss + STYLE_WEIGHT * style_loss

    return content_loss, style_loss, total_loss

def _create_summary(model):
    """ Create summary ops necessary
        Hint: don't forget to merge them
    """
    with tf.name_scope('summaries'):
        tf.summary.scalar('content_loss', model['content_loss'])
        tf.summary.scalar('style_loss', model['style_loss'])
        tf.summary.scalar('total_loss', model['total_loss'])
        tf.summary.histogram('histogram_content_loss', model['content_loss'])
        tf.summary.histogram('histogram_style_loss', model['style_loss'])
        tf.summary.histogram('histogram_total_loss', model['total_loss'])
        return tf.summary.merge_all()

def train(model, generated_image, initial_image):
    """ Train your model.
    Don't forget to create folders for checkpoints and outputs.
    """
    skip_step = 1
    with tf.Session() as sess:
        saver = tf.train.Saver()
        sess.run(tf.global_variables_initializer())
        writer = tf.summary.FileWriter('graphs', sess.graph)
        sess.run(generated_image.assign(initial_image))
        ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/checkpoint'))
        if ckpt and ckpt.model_checkpoint_path and MODEL_TYPE == VGG:
            saver.restore(sess, ckpt.model_checkpoint_path)
        initial_step = model['global_step'].eval()
        
        start_time = time.time()
        for index in range(initial_step, ITERS):
            if index >= 5 and index < 20:
                skip_step = 10
            elif index >= 20:
                skip_step = 20
            
            sess.run(model['optimizer'])
            if (index + 1) % skip_step == 0:
                gen_image, total_loss, summary = sess.run([generated_image, model['total_loss'], 
                                                             model['summary_op']])
                gen_image = gen_image + MEAN_PIXELS
                writer.add_summary(summary, global_step=index)
                print('Step {}\n   Sum: {:5.1f}'.format(index + 1, np.sum(gen_image)))
                print('   Loss: {:5.1f}'.format(total_loss))
                print('   Time: {}'.format(time.time() - start_time))
                start_time = time.time()

                filename = 'outputs/%d.png' % (index)
                save_image(filename, gen_image)

                if (index + 1) % 20 == 0:
                    saver.save(sess, 'checkpoints/style_transfer', index)

reset_graph()
shutil.rmtree('checkpoints')
    
with tf.variable_scope('input') as scope:
    # use variable instead of placeholder because we're training the intial image to make it
    # look like both the content image and the style image
    input_image = tf.Variable(np.zeros([1, IMAGE_HEIGHT, IMAGE_WIDTH, 3]), dtype=tf.float32)
    
make_dir('checkpoints')
make_dir('outputs')
model = load_model(MODEL_TYPE, input_image)
model['global_step'] = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')

content_image = get_resized_image(CONTENT_IMAGE, IMAGE_HEIGHT, IMAGE_WIDTH)
content_image = content_image - MEAN_PIXELS
style_image = get_resized_image(STYLE_IMAGE, IMAGE_HEIGHT, IMAGE_WIDTH)
style_image = style_image - MEAN_PIXELS

if (MODEL_TYPE == GOOGLENET):
    # normalize between -1 and 1
    content_image[0] /= 255
    style_image[0] /= 255

model['content_loss'], model['style_loss'], model['total_loss'] = _create_losses(model, 
                                                input_image, content_image, style_image)
model['optimizer'] = tf.train.AdamOptimizer(LR).minimize(model['total_loss'], 
                                                        global_step=model['global_step'])
model['summary_op'] = _create_summary(model)

initial_image = generate_noise_image(content_image, IMAGE_HEIGHT, IMAGE_WIDTH, NOISE_RATIO)
train(model, input_image, initial_image)

Loading VGG-19
vgg pre-trained model ready
Step 1
   Sum: 24428528.1
   Loss: 2576478464.0
   Time: 1.59613585472
Step 2
   Sum: 24414973.7
   Loss: 2328004096.0
   Time: 0.263147830963
Step 3
   Sum: 24400475.5
   Loss: 2095708032.0
   Time: 0.253443002701
Step 4
   Sum: 24384749.9
   Loss: 1890530944.0
   Time: 0.25806593895
Step 5
   Sum: 24367391.8
   Loss: 1713330560.0
   Time: 0.258044958115
Step 10
   Sum: 24267725.8
   Loss: 1090875648.0
   Time: 0.938369989395
Step 20
   Sum: 24081639.0
   Loss: 581727232.0
   Time: 1.79849410057
Step 40
   Sum: 23888411.7
   Loss: 298144288.0
   Time: 3.84031200409
Step 60
   Sum: 23749657.0
   Loss: 202713632.0
   Time: 3.94758296013
Step 80
   Sum: 23627306.2
   Loss: 154969824.0
   Time: 3.84605789185
Step 100
   Sum: 23514656.6
   Loss: 125804680.0
   Time: 3.85366106033
Step 120
   Sum: 23409940.4
   Loss: 105774808.0
   Time: 3.85711598396
Step 140
   Sum: 23311147.9
   Loss: 91254824.0
   Time: 3.88381385803
Step 160
   Sum: 23219433.5